# SGLang Native APIs

Apart from the OpenAI compatible APIs, the SGLang Runtime also provides its native server APIs. We introduce these following APIs:

- `/generate` (text generation model)
- `/get_model_info`
- `/get_server_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/update_weights`
- `/encode`(embedding model)
- `/v1/rerank`(cross encoder rerank model)
- `/classify`(reward model)
- `/start_expert_distribution_record`
- `/stop_expert_distribution_record`
- `/dump_expert_distribution_record`

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch A Server

In [1]:
import requests
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0"
)
## To run qwen2.5-0.5b-instruct model on the Ascend-Npu, you can execute the following command:
# server_process, port = launch_server_cmd(
#     "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0 --device npu --tp 2 --attention-backend torch_native"
# )

wait_for_server(f"http://localhost:{port}")

[2025-08-03 00:11:49] server_args=ServerArgs(model_path='qwen/qwen2.5-0.5b-instruct', tokenizer_path='qwen/qwen2.5-0.5b-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=32010, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=23652410, con

[2025-08-03 00:11:50] Using default HuggingFace chat template with detected content format: string


[2025-08-03 00:12:01] Attention backend not explicitly specified. Use fa3 backend by default.
[2025-08-03 00:12:01] Init torch distributed begin.


[2025-08-03 00:12:03] Init torch distributed ends. mem usage=0.00 GB


[2025-08-03 00:12:04] Load weight begin. avail mem=77.57 GB


...

[2025-08-03 00:12:06] KV Cache is allocated. #tokens: 20480, K size: 0.12 GB, V size: 0.12 GB
[2025-08-03 00:12:06] Memory pool end. avail mem=47.95 GB


[2025-08-03 00:12:07] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=46.47 GB


[2025-08-03 00:12:08] INFO:     Started server process [881602]
[2025-08-03 00:12:08] INFO:     Waiting for application startup.
[2025-08-03 00:12:08] INFO:     Application startup complete.
[2025-08-03 00:12:08] INFO:     Uvicorn running on http://0.0.0.0:32010 (Press CTRL+C to quit)


[2025-08-03 00:12:09] INFO:     127.0.0.1:50662 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-03 00:12:09] INFO:     127.0.0.1:50678 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-03 00:12:09] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 00:12:10] INFO:     127.0.0.1:50688 - "POST /generate HTTP/1.1" 200 OK
[2025-08-03 00:12:10] The server is fired up and ready to roll!


## Generate (text generation model)
Generate completions. This is similar to the `/v1/completions` in OpenAI API. Detailed parameters can be found in the [sampling parameters](./sampling_params.md).

In [2]:
url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-08-03 00:12:14] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 00:12:14] Decode batch. #running-req: 1, #token: 40, token usage: 0.00, cuda graph: False, gen throughput (token/s): 5.91, #queue-req: 0, 


[2025-08-03 00:12:14] Decode batch. #running-req: 1, #token: 80, token usage: 0.00, cuda graph: False, gen throughput (token/s): 123.06, #queue-req: 0, 


[2025-08-03 00:12:15] Decode batch. #running-req: 1, #token: 120, token usage: 0.01, cuda graph: False, gen throughput (token/s): 127.17, #queue-req: 0, 
[2025-08-03 00:12:15] INFO:     127.0.0.1:50702 - "POST /generate HTTP/1.1" 200 OK


## Get Model Info

Get the information of the model.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.
- `tokenizer_path`: The path/name of the tokenizer.
- `preferred_sampling_params`: The default sampling params specified via `--preferred-sampling-params`. `None` is returned in this example as we did not explicitly configure it in server args.

In [3]:
url = f"http://localhost:{port}/get_model_info"

response = requests.get(url)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json["is_generation"] is True
assert response_json["tokenizer_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json["preferred_sampling_params"] is None
assert response_json.keys() == {
    "model_path",
    "is_generation",
    "tokenizer_path",
    "preferred_sampling_params",
}

[2025-08-03 00:12:15] INFO:     127.0.0.1:50704 - "GET /get_model_info HTTP/1.1" 200 OK


## Get Server Info
Gets the server information including CLI arguments, token limits, and memory pool sizes.
- Note: `get_server_info` merges the following deprecated endpoints:
  - `get_server_args`
  - `get_memory_pool_size` 
  - `get_max_total_num_tokens`

In [4]:
# get_server_info

url = f"http://localhost:{port}/get_server_info"

response = requests.get(url)
print_highlight(response.text)

[2025-08-03 00:12:15] INFO:     127.0.0.1:50708 - "GET /get_server_info HTTP/1.1" 200 OK


## Health Check
- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [5]:
url = f"http://localhost:{port}/health_generate"

response = requests.get(url)
print_highlight(response.text)

[2025-08-03 00:12:15] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 00:12:16] INFO:     127.0.0.1:50714 - "GET /health_generate HTTP/1.1" 200 OK


In [6]:
url = f"http://localhost:{port}/health"

response = requests.get(url)
print_highlight(response.text)

[2025-08-03 00:12:16] INFO:     127.0.0.1:50716 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = f"http://localhost:{port}/flush_cache"

response = requests.post(url)
print_highlight(response.text)

[2025-08-03 00:12:16] Cache flushed successfully!
[2025-08-03 00:12:16] INFO:     127.0.0.1:50720 - "POST /flush_cache HTTP/1.1" 200 OK


## Update Weights From Disk

Update model weights from disk without restarting the server. Only applicable for models with the same architecture and parameter size.

SGLang support `update_weights_from_disk` API for continuous evaluation during training (save checkpoint to disk and update weights from disk).


In [8]:
# successful update with same architecture and size

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] is True
assert response.json()["message"] == "Succeeded to update model weights."

[2025-08-03 00:12:16] Start update_weights. Load format=auto
[2025-08-03 00:12:16] Update engine weights online from disk begin. avail mem=41.77 GB
[2025-08-03 00:12:16] Using model weights format ['*.safetensors']


[2025-08-03 00:12:16] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.59it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.58it/s]



[2025-08-03 00:12:16] Update weights end.
[2025-08-03 00:12:16] Cache flushed successfully!
[2025-08-03 00:12:16] INFO:     127.0.0.1:50736 - "POST /update_weights_from_disk HTTP/1.1" 200 OK


In [9]:
# failed update with different parameter size or wrong name

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct-wrong"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] is False
assert response_json["message"] == (
    "Failed to get weights iterator: "
    "qwen/qwen2.5-0.5b-instruct-wrong"
    " (repository not found)."
)

[2025-08-03 00:12:16] Start update_weights. Load format=auto
[2025-08-03 00:12:16] Update engine weights online from disk begin. avail mem=41.50 GB
[2025-08-03 00:12:16] Failed to get weights iterator: qwen/qwen2.5-0.5b-instruct-wrong (repository not found).
[2025-08-03 00:12:16] INFO:     127.0.0.1:50750 - "POST /update_weights_from_disk HTTP/1.1" 400 Bad Request


In [10]:
terminate_process(server_process)

## Encode (embedding model)

Encode text into embeddings. Note that this API is only available for [embedding models](openai_api_embeddings.html#openai-apis-embedding) and will raise an error for generation models.
Therefore, we launch a new server to server an embedding model.

In [11]:
embedding_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-1.5B-instruct \
    --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-03 00:12:24] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=True, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=38291, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_s

[2025-08-03 00:12:24] Using default HuggingFace chat template with detected content format: string


[2025-08-03 00:12:32] Downcasting torch.float32 to torch.float16.


[2025-08-03 00:12:32] Overlap scheduler is disabled for embedding models.
[2025-08-03 00:12:32] Downcasting torch.float32 to torch.float16.
[2025-08-03 00:12:32] Attention backend not explicitly specified. Use fa3 backend by default.
[2025-08-03 00:12:32] Init torch distributed begin.


[2025-08-03 00:12:33] Init torch distributed ends. mem usage=0.00 GB


...

[2025-08-03 00:12:38] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=57.64 GB


[2025-08-03 00:12:38] INFO:     Started server process [884660]
[2025-08-03 00:12:38] INFO:     Waiting for application startup.
[2025-08-03 00:12:38] INFO:     Application startup complete.
[2025-08-03 00:12:38] INFO:     Uvicorn running on http://0.0.0.0:38291 (Press CTRL+C to quit)


[2025-08-03 00:12:39] INFO:     127.0.0.1:42032 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-03 00:12:39] INFO:     127.0.0.1:41660 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-03 00:12:39] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 00:12:40] INFO:     127.0.0.1:41668 - "POST /encode HTTP/1.1" 200 OK
[2025-08-03 00:12:40] The server is fired up and ready to roll!


In [12]:
# successful encode for embedding model

url = f"http://localhost:{port}/encode"
data = {"model": "Alibaba-NLP/gte-Qwen2-1.5B-instruct", "text": "Once upon a time"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(f"Text embedding (first 10): {response_json['embedding'][:10]}")

[2025-08-03 00:12:44] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-03 00:12:44] INFO:     127.0.0.1:41684 - "POST /encode HTTP/1.1" 200 OK


In [13]:
terminate_process(embedding_process)

## v1/rerank (cross encoder rerank model)
Rerank a list of documents given a query using a cross-encoder model. Note that this API is only available for cross encoder model like [BAAI/bge-reranker-v2-m3](https://huggingface.co/BAAI/bge-reranker-v2-m3) with `attention-backend` `triton` and `torch_native`.


In [14]:
reranker_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path BAAI/bge-reranker-v2-m3 \
    --host 0.0.0.0 --disable-radix-cache --chunked-prefill-size -1 --attention-backend triton --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-03 00:12:51] server_args=ServerArgs(model_path='BAAI/bge-reranker-v2-m3', tokenizer_path='BAAI/bge-reranker-v2-m3', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=True, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=36587, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=220027887, constrained

[2025-08-03 00:12:52] No chat template found, defaulting to 'string' content format


[2025-08-03 00:13:00] Downcasting torch.float32 to torch.float16.


[2025-08-03 00:13:01] Overlap scheduler is disabled for embedding models.
[2025-08-03 00:13:01] Downcasting torch.float32 to torch.float16.
[2025-08-03 00:13:01] Init torch distributed begin.


[2025-08-03 00:13:02] Init torch distributed ends. mem usage=0.00 GB


...

[2025-08-03 00:13:05] max_total_num_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, max_running_requests=200, context_len=8194, available_gpu_mem=58.24 GB


[2025-08-03 00:13:06] INFO:     Started server process [887426]
[2025-08-03 00:13:06] INFO:     Waiting for application startup.
[2025-08-03 00:13:06] INFO:     Application startup complete.
[2025-08-03 00:13:06] INFO:     Uvicorn running on http://0.0.0.0:36587 (Press CTRL+C to quit)


[2025-08-03 00:13:07] INFO:     127.0.0.1:47602 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-03 00:13:07] INFO:     127.0.0.1:47606 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-03 00:13:07] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 00:13:08] INFO:     127.0.0.1:47610 - "POST /encode HTTP/1.1" 200 OK
[2025-08-03 00:13:08] The server is fired up and ready to roll!


In [15]:
# compute rerank scores for query and documents

url = f"http://localhost:{port}/v1/rerank"
data = {
    "model": "BAAI/bge-reranker-v2-m3",
    "query": "what is panda?",
    "documents": [
        "hi",
        "The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.",
    ],
}

response = requests.post(url, json=data)
response_json = response.json()
for item in response_json:
    print_highlight(f"Score: {item['score']:.2f} - Document: '{item['document']}'")

[2025-08-03 00:13:12] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-03 00:13:12] Prefill batch. #new-seq: 1, #new-token: 43, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-03 00:13:12] INFO:     127.0.0.1:34618 - "POST /v1/rerank HTTP/1.1" 200 OK


In [16]:
terminate_process(reranker_process)

## Classify (reward model)

SGLang Runtime also supports reward models. Here we use a reward model to classify the quality of pairwise generations.

In [17]:
# Note that SGLang now treats embedding models and reward models as the same type of models.
# This will be updated in the future.

reward_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Skywork/Skywork-Reward-Llama-3.1-8B-v0.2 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-03 00:13:19] server_args=ServerArgs(model_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=True, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=30404, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False

[2025-08-03 00:13:20] Using default HuggingFace chat template with detected content format: string


[2025-08-03 00:13:27] Overlap scheduler is disabled for embedding models.
[2025-08-03 00:13:27] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-03 00:13:27] Init torch distributed begin.


[2025-08-03 00:13:30] Init torch distributed ends. mem usage=-33.45 GB


[2025-08-03 00:13:31] Load weight begin. avail mem=56.38 GB


...

Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.21it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.57it/s]

[2025-08-03 00:13:34] Load weight end. type=LlamaForSequenceClassification, dtype=torch.bfloat16, avail mem=64.46 GB, mem usage=-8.07 GB.
[2025-08-03 00:13:34] KV Cache is allocated. #tokens: 20480, K size: 1.25 GB, V size: 1.25 GB
[2025-08-03 00:13:34] Memory pool end. avail mem=61.66 GB


[2025-08-03 00:13:35] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=61.09 GB


[2025-08-03 00:13:35] INFO:     Started server process [890053]
[2025-08-03 00:13:35] INFO:     Waiting for application startup.
[2025-08-03 00:13:35] INFO:     Application startup complete.
[2025-08-03 00:13:35] INFO:     Uvicorn running on http://0.0.0.0:30404 (Press CTRL+C to quit)


[2025-08-03 00:13:36] INFO:     127.0.0.1:40742 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-03 00:13:36] INFO:     127.0.0.1:40758 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-03 00:13:36] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [18]:
from transformers import AutoTokenizer

PROMPT = (
    "What is the range of the numeric output of a sigmoid node in a neural network?"
)

RESPONSE1 = "The output of a sigmoid node is bounded between -1 and 1."
RESPONSE2 = "The output of a sigmoid node is bounded between 0 and 1."

CONVS = [
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE1}],
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE2}],
]

tokenizer = AutoTokenizer.from_pretrained("Skywork/Skywork-Reward-Llama-3.1-8B-v0.2")
prompts = tokenizer.apply_chat_template(CONVS, tokenize=False)

url = f"http://localhost:{port}/classify"
data = {"model": "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2", "text": prompts}

responses = requests.post(url, json=data).json()
for response in responses:
    print_highlight(f"reward: {response['embedding'][0]}")

[2025-08-03 00:14:41] Prefill batch. #new-seq: 2, #new-token: 136, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-03 00:14:41] INFO:     127.0.0.1:40768 - "POST /encode HTTP/1.1" 200 OK
[2025-08-03 00:14:41] The server is fired up and ready to roll!


[2025-08-03 00:14:55] INFO:     127.0.0.1:52182 - "POST /classify HTTP/1.1" 200 OK


In [19]:
terminate_process(reward_process)

## Capture expert selection distribution in MoE models

SGLang Runtime supports recording the number of times an expert is selected in a MoE model run for each expert in the model. This is useful when analyzing the throughput of the model and plan for optimization.

*Note: We only print out the first 10 lines of the csv below for better readability. Please adjust accordingly if you want to analyze the results more deeply.*

In [20]:
expert_record_server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path Qwen/Qwen1.5-MoE-A2.7B --host 0.0.0.0 --expert-distribution-recorder-mode stat"
)

wait_for_server(f"http://localhost:{port}")

[2025-08-03 00:15:03] server_args=ServerArgs(model_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=35208, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=416034830, constraine

[2025-08-03 00:15:03] Using default HuggingFace chat template with detected content format: string


[2025-08-03 00:15:11] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-03 00:15:11] Init torch distributed begin.


[2025-08-03 00:15:11] Init torch distributed ends. mem usage=0.00 GB


[2025-08-03 00:15:13] Load weight begin. avail mem=60.24 GB


...

Loading safetensors checkpoint shards: 100% Completed | 8/8 [00:06<00:00,  1.43it/s]
Loading safetensors checkpoint shards: 100% Completed | 8/8 [00:06<00:00,  1.25it/s]

[2025-08-03 00:15:20] Load weight end. type=Qwen2MoeForCausalLM, dtype=torch.bfloat16, avail mem=33.42 GB, mem usage=26.81 GB.
[2025-08-03 00:15:20] KV Cache is allocated. #tokens: 20480, K size: 1.88 GB, V size: 1.88 GB
[2025-08-03 00:15:20] Memory pool end. avail mem=29.51 GB


[2025-08-03 00:15:20] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=28.94 GB


[2025-08-03 00:15:21] INFO:     Started server process [897212]
[2025-08-03 00:15:21] INFO:     Waiting for application startup.
[2025-08-03 00:15:21] INFO:     Application startup complete.
[2025-08-03 00:15:21] INFO:     Uvicorn running on http://0.0.0.0:35208 (Press CTRL+C to quit)


[2025-08-03 00:15:22] INFO:     127.0.0.1:33842 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-03 00:15:22] INFO:     127.0.0.1:33844 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-03 00:15:22] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 00:15:22] Using default MoE kernel config. Performance might be sub-optimal! Config file not found at /public_sglang_ci/runner-l1c-gpu-1/_work/sglang/sglang/python/sglang/srt/layers/moe/fused_moe_triton/configs/triton_3_3_1/E=60,N=1408,device_name=NVIDIA_H100_80GB_HBM3.json, you can create them with https://github.com/sgl-project/sglang/tree/main/benchmark/kernels/fused_moe_triton


In [21]:
response = requests.post(f"http://localhost:{port}/start_expert_distribution_record")
print_highlight(response)

url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

response = requests.post(f"http://localhost:{port}/stop_expert_distribution_record")
print_highlight(response)

response = requests.post(f"http://localhost:{port}/dump_expert_distribution_record")
print_highlight(response)

[2025-08-03 00:15:36] Resetting ExpertDistributionRecorder...
[2025-08-03 00:15:36] INFO:     127.0.0.1:33876 - "POST /start_expert_distribution_record HTTP/1.1" 200 OK


[2025-08-03 00:15:36] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-08-03 00:15:37] INFO:     127.0.0.1:33860 - "POST /generate HTTP/1.1" 200 OK
[2025-08-03 00:15:37] The server is fired up and ready to roll!
[2025-08-03 00:15:37] INFO:     127.0.0.1:41030 - "POST /generate HTTP/1.1" 200 OK


[2025-08-03 00:15:37] INFO:     127.0.0.1:41036 - "POST /stop_expert_distribution_record HTTP/1.1" 200 OK


[2025-08-03 00:15:38] Write expert distribution to /tmp/expert_distribution_recorder_1754180138.2498662.pt
[2025-08-03 00:15:38] Resetting ExpertDistributionRecorder...
[2025-08-03 00:15:38] INFO:     127.0.0.1:41044 - "POST /dump_expert_distribution_record HTTP/1.1" 200 OK


In [22]:
terminate_process(expert_record_server_process)